<a href="https://colab.research.google.com/github/vash4/My-Project/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Cell-1
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)


In [ ]:
#Cell-2
!git clone https://github.com/NVlabs/stylegan2.git


In [ ]:
#Cell-3
!python /content/stylegan2/run_generator.py generate-images \
--network=gdrive:networks/stylegan2-ffhq-config-f.pkl \
 --seeds=1-50 --truncation-psi=0.5


In [ ]:
#Cell-4
!ffmpeg -r 3 -i /content/results/00000-generate-images/seed%04d.png -vcodec mpeg4 -y people.mp4


In [ ]:
#Cell-5
!cp people.mp4 /content


In [ ]:
#Cell-6
import sys
sys.path.insert(0, "/content/stylegan2")


In [ ]:
#Cell-7
import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
import pretrained_networks


In [ ]:
#Cell-8
def seeder(seeds, vector_size):
 result = []
 for seed in seeds:
   rnd = np.random.RandomState(seed)
   result.append( rnd.randn(1, vector_size))
 return result



In [ ]:
#Cell-9
def generate_images(gs, seeds, truncation_psi):
   noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
 
   Gs_kwargs = dnnlib.EasyDict()
   Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
   Gs_kwargs.randomize_noise = False
   if truncation_psi is not None:
       Gs_kwargs.truncation_psi = truncation_psi
 
   for seed_idx, seed in enumerate(seeds):
       print('Generating image for seed %d/%d ...' % (seed_idx, len(seeds)))
       rnd = np.random.RandomState()
       tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
       images = Gs.run(seed, None, **Gs_kwargs) # [minibatch, height, width, channel]
       path = f"/content/out/image{seed_idx}.png"
       PIL.Image.fromarray(images[0], 'RGB').save(path)





In [ ]:
#Cell-10
network_pkl = "gdrive:networks/stylegan2-ffhq-config-f.pkl"
print('Loading networks from "%s"...' % network_pkl)
Gs = pretrained_networks.load_networks(network_pkl)


In [ ]:
#Cell-11
vector_size = Gs.input_shape[1:][0]
seeds = seeder(range(6, 5), vector_size)





In [ ]:
#Cell-12
!mkdir /content/out



mkdir: cannot create directory ‘/content/out’: File exists


In [1]:
#Cell-13
seeds = seeder([6, 5], vector_size)


NameError: ignored

In [ ]:
#Cell-14
STEPS = 300
diff = seeds[1] - seeds[0]
step = diff / STEPS
current = seeds[0].copy()
 
seeds2 = []
for i in range(STEPS):
 seeds2.append(current)
 current = current + step


In [ ]:
#Cell-15
generate_images(Gs, seeds2, truncation_psi=0.01)


In [ ]:
#Cell-16
!ffmpeg -r 50 -i /content/out/image%d.png -vcodec mpeg4 -y faces.mp4


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
#Cell-17
!cp faces.mp4 /content
